# MER_AdviesScraper

Doel: MER adviezen uitgebracht vanaf 2019 tot en met 2024 verzamelen.

Toepassing: Bachelor-scriptie Informatiekunde, Universiteit van Amsterdam

Contact: mitchell.malaihollo@student.uva.nl

Metadata Formatering volgens de [Woogle metadata specificatie](https://github.com/wooverheid/WoogleDocumentatie/tree/master/SPEC%20MetadataSchema) 

In [10]:
from openai import OpenAI
import json
import time
from dotenv import load_dotenv
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin

In [2]:
USER_AGENT = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) "
    "Gecko/20100101 Firefox/124.0 "
    "AdviescollegeScraper (+mailto:14605120@student.uva.nl; "
    "voor BSc-scriptie Informatiekunde UvA; "
    "doel: informatie inwinnen over verschillende adviescolleges)"
)

env_path = r"C:\Users\31633\OneDrive\Bureaublad\OpenAI_API_Key.env"  
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")
print("Key geladen:", api_key is not None)

Key geladen: True


In [3]:
def beginSelenium():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"user-agent={USER_AGENT}")
    
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    return driver

def retrieveLinksPagina(driver, url):
    driver.get(url)
    
    jaar = url[-4:]
    
    link_elementen = driver.find_elements(By.TAG_NAME, "a")
    urls = []
    
    for element in link_elementen:
        href = element.get_attribute("href")
    
        if href and f"{jaar}/advies/" in href:
            urls.append(href)
    
    return urls


alle_jaren_urls = []
    
jaren = [2019, 2020, 2021, 2022, 2023, 2024]
    
for jaar in jaren:
    url = f"https://www.rli.nl/onze-adviezen?f%5B0%5D=jaar%3A{jaar}"
    driver = beginSelenium()
    urls = retrieveLinksPagina(driver, url)
    alle_jaren_urls.extend(urls)
    driver.quit()
    time.sleep(5)

In [4]:
print("\nAlle unieke adviezen links:")
for link in alle_jaren_urls:
    print(link)

print(f"\nTotaal aantal unieke adviezen-links: {len(alle_jaren_urls)}")


Alle unieke adviezen links:
https://www.rli.nl/publicaties/2019/advies/naar-een-duurzame-economie
https://www.rli.nl/publicaties/2019/advies/waardevol-toerisme
https://www.rli.nl/publicaties/2019/advies/europees-landbouwbeleid
https://www.rli.nl/publicaties/2019/advies/luchtvaartbeleid
https://www.rli.nl/publicaties/2020/advies/toegang-tot-de-stad-hoe-publieke-voorzieningen-wonen-en-vervoer-de-sleutel-voor-burgers-vormen
https://www.rli.nl/publicaties/2020/advies/stop-bodemdaling-in-veenweidegebieden-het-groene-hart-als-voorbeeld
https://www.rli.nl/publicaties/2020/advies/groen-uit-de-crisis
https://www.rli.nl/publicaties/2020/advies/verzet-de-wissel-naar-beter-internationaal-reizigersvervoer-per-trein
https://www.rli.nl/publicaties/2020/advies/de-bodem-bereikt
https://www.rli.nl/publicaties/2020/advies/greep-op-gevaarlijke-stoffen
https://www.rli.nl/publicaties/2021/advies/boeren-met-toekomst
https://www.rli.nl/publicaties/2021/advies/geef-richting-maak-ruimte
https://www.rli.nl/publ

In [5]:
client = OpenAI()
res = []

for url in alle_jaren_urls:
    prompt = f"""
    Lees dit document: {url}

    Vul de onderstaande JSON in volgens https://github.com/wooverheid/WoogleDocumentatie/tree/master/SPEC%20MetadataSchema:

    {{
        "dc_title": "",
        "foi_publishedDate": "",
        "dc_description": ""
    }}

    Regels:
    - Alleen écht gevonden info invullen.
    - Max 100 woorden in dc_description.
    - Ontbreekt iets? dan "" laten.
    - De foi_publishedDate staat bij de url onder het kopje Datum van publicatie.
    - datum in jjjj-mm-dd
    - Output = alleen geldige JSON niks anders.
    """
    response = client.responses.create(
        model="gpt-5.1",
        input=prompt,
        tools=[{"type": "web_search"}],
        max_output_tokens=800,
    )
    
    json_str = response.output_text.strip()

    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print("Kon JSON niet parsen voor URL:", url)
        print("Ruwe output was:\n", json_str)
        continue

    data["dc_source"] = url
    data["dc_publisher"] = "oorg10261"
    data["dc_type"] = "2e"
        
    res.append(data)
    
    print(json.dumps(data, ensure_ascii=False, indent=2))
    
    time.sleep(12)

with open("RLI_adviesScraper_data1.json", "w", encoding="utf-8") as f:
    json.dump(res, f, ensure_ascii=False, indent=2)
    
print("\nAlle JSON-data opgeslagen in RLI_adviesScraper_data1.json")

{
  "dc_title": "Naar een duurzame economie",
  "foi_publishedDate": "2019-12-10",
  "dc_description": "Advies van de Raad voor de leefomgeving en infrastructuur over hoe de rijksoverheid kan sturen op de transitie naar een duurzame economie. Het rapport past wetenschappelijke inzichten over transitiesturing toe op drie dossiers: de energietransitie, de grondstoffentransitie en de voedseltransitie, en bevat aandachtspunten en aanbevelingen voor overheidsbeleid.",
  "dc_source": "https://www.rli.nl/publicaties/2019/advies/naar-een-duurzame-economie",
  "dc_publisher": "oorg10261",
  "dc_type": "2e"
}
{
  "dc_title": "Waardevol toerisme",
  "foi_publishedDate": "2019-09-06",
  "dc_description": "Advies van de Raad voor de leefomgeving en infrastructuur (Rli) over hoe te sturen op een gebalanceerde groei van inkomend en binnenlands toerisme in Nederland. Het rapport benadrukt de noodzaak van een perspectiefwisseling in toerismebeleid, met aandacht voor zowel economische opbrengsten als ef

In [6]:
with open("RLI_adviesScraper_data1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Aantal items in JSON bestand: " + str(len(data)))

Aantal items in JSON bestand: 30


In [12]:
res2 = []

with open("RLI_adviesScraper_data1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def retrieveLinksPagina2(driver, urltje):
    driver.get(urltje)
        
    link_elementen = driver.find_elements(By.TAG_NAME, "a")
    urls = {
        "advies": None,
        "samenvatting": None,
        "kabinetsreactie": None,
        "persbericht": None,
    }
    
    for element in link_elementen:
        href = element.get_attribute("href")
        if not href:
            continue

        href_abs = urljoin(urltje, href)

        text = (element.text or "").strip()
        text_lower = text.lower()
        class_attr = (element.get_attribute("class") or "").lower()
        title_attr = (element.get_attribute("title") or "").lower()
        href_lower = href_abs.lower()

        if urls["kabinetsreactie"] is None and (
            "kabinetsreactie" in href_lower
            or text_lower.startswith("kabinetsreactie")
        ):
            urls["kabinetsreactie"] = href_abs
            continue

        if urls["persbericht"] is None and (
            "/pers/" in href_abs
            or text_lower == "persbericht"
        ):
            urls["persbericht"] = href_abs
            continue

        if urls["advies"] is None and (
            "advies_" in href_lower and href_lower.endswith(".pdf")
        ):
            urls["advies"] = href_abs
            continue

        if urls["samenvatting"] is None and (
            "adview=samenvatting" in href_lower
            or "adview-samenvatting" in class_attr
            or title_attr == "samenvatting"
        ):
            urls["samenvatting"] = href_abs
            continue

    return urls    

driver = beginSelenium()

for item in data:
    dc_source2 = item.get("dc_source")

    urltjes = retrieveLinksPagina2(driver, dc_source2)

    foi_files = []
    for key, value in urltjes.items():
        if not value:
            continue
        foi_files.append({
            "foi_title": key,
            "foi_source": value,
        })

    res2.append({
        "dc_title": item.get("dc_title"),
        "foi_publishedDate": item.get("foi_publishedDate"),
        "dc_description": item.get("dc_description"),
        "dc_source": dc_source2,
        "dc_publisher": item.get("dc_publisher"),
        "dc_type": item.get("dc_type"),
        "foi_files": foi_files
    })

    time.sleep(5)

driver.quit()

with open("RLI_adviesScraper_data2.json", "w", encoding="utf-8") as f:
    json.dump(res2, f, ensure_ascii=False, indent=2)

print("\nAlle JSON-data opgeslagen in RLI_adviesScraper_data2.json")


Alle JSON-data opgeslagen in RLI_adviesScraper_data2.json


In [13]:
with open("RLI_adviesScraper_data2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Aantal items in JSON bestand: " + str(len(data)))

Aantal items in JSON bestand: 30
